In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Zadanie 4.1

In [ ]:
I = cv2.imread('../data/images/I.jpg')
I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
J = cv2.imread('../data/images/J.jpg')
J = cv2.cvtColor(J, cv2.COLOR_BGR2GRAY)

difference = cv2.absdiff(I, J)

In [ ]:
plt.subplots(1, 3, figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(I, 'gray')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(J, 'gray')
plt.axis('off')
plt.subplot(1, 3, 3)
plt.imshow(difference, 'gray')
plt.axis('off')
plt.show()

In [ ]:
def block_optical_flow(I, J, W2, dX, dY):
    X, Y = I.shape
    u = np.zeros(I.shape)
    v = np.zeros(I.shape)
    for j in range(W2 * 2, X - (W2 * 2)):
        for i in range(W2 * 2, Y - (W2 * 2)):
            IO = I[j-W2:j+W2+1, i-W2:i+W2+1].astype(float)

            min_distance = np.inf
            for y in range(-dX, dX + 1):
                for x in range(-dY, dY + 1):
                        JO = J[j-W2+y:j+W2+y+1, i-W2+x:i-W2+x+1].astype(float)
                        distance = np.sqrt(np.sum(np.square(JO - IO)))
                        if distance < min_distance:
                            min_distance = distance
                            u[j, i] = x
                            v[j, i] = y
    return u, v

def vis_flow(u, v):
    magnitude, angle = cv2.cartToPolar(u, v)
    magnitude_normalized = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    HSV = np.zeros((u.shape[0], u.shape[1], 3), dtype=np.uint8)
    HSV[:,:,0] = (angle * 90 / np.pi).astype(np.uint8)
    HSV[:,:,1] = magnitude_normalized.astype(np.uint8)
    HSV[:,:,2] = np.uint8(255)

    RGB = cv2.cvtColor(HSV, cv2.COLOR_HSV2RGB)

    plt.imshow(RGB)
    plt.show()

In [ ]:
W2 = dX = dY = 3

In [ ]:
u, v = block_optical_flow(I, J, W2, dX, dY)

vis_flow(u, v)

#### Inna sekwencja obrazów

In [ ]:
I2 = cv2.imread('../data/images/cm1.png')
I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2GRAY)
J2 = cv2.imread('../data/images/cm2.png')
J2 = cv2.cvtColor(J2, cv2.COLOR_BGR2GRAY)

In [ ]:
W2 = dX = dY = 3

In [ ]:
u2, v2 = block_optical_flow(I2, J2, W2, dX, dY)

vis_flow(u2, v2)

## Zadanie 4.2

In [ ]:
def pyramid(im, max_scale):
    images =[im]
    for k in range(1, max_scale):
        images.append(cv2.resize(images[k - 1], (0, 0), fx=0.5, fy=0.5))
    return images

In [ ]:
def of(I, J, W2=3, dY=3, dX=3):
    X, Y = I.shape
    u = np.zeros(I.shape)
    v = np.zeros(I.shape)

    for j in range(W2 * 2, X - (W2 * 2)):
        for i in range(W2 * 2, Y - (W2 * 2)):
            IO = I[j-W2:j+W2+1, i-W2:i+W2+1].astype(float)

            min_distance = np.inf
            for y in range(-dX, dX + 1):
                for x in range(-dY, dY + 1):
                        JO = J[j-W2+y:j+W2+y+1, i-W2+x:i-W2+x+1].astype(float)
                        distance = np.sqrt(np.sum(np.square(JO - IO)))
                        if distance < min_distance:
                            min_distance = distance
                            u[j, i] = x
                            v[j, i] = y
    return u, v

In [ ]:
I_pyramid = pyramid(I, 4)
J_pyramid = pyramid(J, 4)

I = I_pyramid[-1]
J = J_pyramid[-1]

for s in range(len(I_pyramid) - 2, -1, -1):
    u, v = of(I_pyramid[s], J_pyramid[s], W2, dY, dX)

    I_new = np.zeros(I.shape, dtype=np.uint8)
    for j in range(I.shape[0]):
        for i in range(I.shape[1]):
            x = min(max(i + u[j, i], 0), I.shape[1] - 1)
            y = min(max(j + v[j, i], 0), I.shape[0] - 1)
            I_new[j, i] = I[int(y), int(x)]
            
    I = cv2.resize(I_new, (0, 0), fx=2, fy=2)
    J = J_pyramid[s]

vis_flow(u, v)

#### Inna sekwencja obrazów

In [ ]:
I_pyramid = pyramid(I2, 4)
J_pyramid = pyramid(J2, 4)

I = I_pyramid[-1]
J = J_pyramid[-1]

for s in range(len(I_pyramid) - 2, -1, -1):
    u, v = of(I_pyramid[s], J_pyramid[s], W2, dY, dX)

    I_new = np.zeros(I.shape, dtype=np.uint8)
    for j in range(I.shape[0]):
        for i in range(I.shape[1]):
            x = min(max(i + u[j, i], 0), I.shape[1] - 1)
            y = min(max(j + v[j, i], 0), I.shape[0] - 1)
            I_new[j, i] = I[int(y), int(x)]
            
    I = cv2.resize(I_new, (0, 0), fx=2, fy=2)
    J = J_pyramid[s]

vis_flow(u, v)